In [277]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#importing libraries

import numpy as np
from pandas import DataFrame
import pandas_datareader as web
import keras
import plotly.graph_objects as go
import tensorflow as tf
import pandas as pd
import itertools
from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense,LSTM
from __future__ import absolute_import,division,print_function
import os
import math

In [133]:
### Importing required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [239]:
## reading the dataset
train_data=pd.read_csv('/content/Japan_cities_rainfall.csv')
to_predict=pd.read_csv('/content/rainfall_to_predict.csv')

# making a copy of original datset 
train_data_original=train_data
to_predict_original=to_predict

In [176]:
train_data.head()

,date_time,place,longitude,latitude,rainfall
0,20200918_080000,Naha,127.686667,26.206667,0.0
1,20200918_080000,Fukuoka,130.375000,33.581667,0.0
2,20200918_080000,Sendai,140.896667,38.261667,0.0
3,20200918_080000,Osaka,135.518333,34.681667,0.0
4,20200918_080000,Nigata,139.018333,37.893333,3.0


In [177]:
train_data.dtypes

date_time     object
place         object
longitude    float64
latitude     float64
rainfall     float64
dtype: object

In [240]:
## we shall make the date_time column to pandas datetime 
train_data['date_time'] = pd.to_datetime(train_data['date_time'],format='%Y%m%d_%H%M%S')

In [179]:
train_data.dtypes
## now date_time has changed to pandas datetime

date_time    datetime64[ns]
place                object
longitude           float64
latitude            float64
rainfall            float64
dtype: object

In [241]:
place_name = [' Tokyo',' Naha',' Sendai',' Osaka',' Nigata',' Fukuoka']

In [242]:

place_list= [
train_data.loc[train_data['place']==' Tokyo'],
train_data.loc[train_data['place']==' Naha'],
train_data.loc[train_data['place']==' Sendai'],
train_data.loc[train_data['place']==' Osaka'],
train_data.loc[train_data['place']==' Nigata'],
train_data.loc[train_data['place']==' Fukuoka']]

In [243]:
for i in place_list:
  i.index = i.date_time
  i.drop(['place','longitude','date_time','latitude'],axis = 1,inplace = True)
  

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [246]:
x=[]
for i in place_list:
  x.append(i.resample('30min').mean())
place_list = x.copy()

,rainfall
date_time,
2020-09-24 09:00:00,1.0
2020-09-24 09:30:00,0.4
2020-09-24 10:00:00,0.0
2020-09-24 10:30:00,NaN
2020-09-24 11:00:00,0.0
2020-09-24 11:30:00,0.0
2020-09-24 12:00:00,0.0
2020-09-24 12:30:00,0.0
2020-09-24 13:00:00,0.0


In [261]:
from pyts.preprocessing import InterpolationImputer
imputer = InterpolationImputer()
for i in range(0,6):
  new=place_list[i].rainfall
  new=imputer.transform([new])
  new = new[0].tolist()
  place_list[i]['rainfall'] = new
  place_list[i].drop('r2',inplace=True,axis=1)

In [268]:
place_list[0][290:300]

,rainfall
date_time,
2020-09-24 09:00:00,1.0
2020-09-24 09:30:00,0.4
2020-09-24 10:00:00,0.0
2020-09-24 10:30:00,0.0
2020-09-24 11:00:00,0.0
2020-09-24 11:30:00,0.0
2020-09-24 12:00:00,0.0
2020-09-24 12:30:00,0.0
2020-09-24 13:00:00,0.0


#data preprocessing for LSTM

In [271]:
from sklearn.preprocessing import MinMaxScaler
place_dataset = []
scaler = MinMaxScaler(feature_range=(0, 1))
for i in place_list:
  dataset = i
  dataset = dataset.values
  dataset = dataset.astype('float32')
  dataset = scaler.fit_transform(dataset)
  place_dataset.append(dataset)

In [272]:
place_train_list=[]
place_test_list=[]
for i in place_dataset:
  dataset = i
  train_size = int(len(dataset) * 0.7)
  test_size = len(dataset) - train_size
  train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
  place_train_list.append(train)
  place_test_list.append(test)


In [273]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [280]:
trainX_list = []
trainY_list=[]
testX_list =[]
testY_list=[]
for i in range(6):
  tX,tY = create_dataset(place_train_list[i])
  testx,testy = create_dataset(place_test_list[i])
  tX=np.reshape(tX, (tX.shape[0], 1, tX.shape[1]))
  testx = np.reshape(testx, (testx.shape[0], 1, testx.shape[1]))
  trainX_list.append(tX)
  trainY_list.append(tY)
  testX_list.append(testx)
  testY_list.append(testy)

In [281]:
trainX_list[0]

array([[[0.]],

       [[0.]],

       [[0.]],

       ...,

       [[0.]],

       [[0.]],

       [[0.]]], dtype=float32)

In [284]:
model_list=[]
for i in range(6):
  model = Sequential()
  model.add(LSTM(4, input_shape=(1, 1)))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error', optimizer='adam')
  model.fit(trainX_list[i], trainY_list[i], epochs=10, batch_size=1, verbose=2)
  model_list.append(model)

Epoch 1/10
2251/2251 - 3s - loss: 0.0104
Epoch 2/10
2251/2251 - 2s - loss: 0.0027
Epoch 3/10
2251/2251 - 2s - loss: 0.0019
Epoch 4/10
2251/2251 - 2s - loss: 0.0019
Epoch 5/10
2251/2251 - 2s - loss: 0.0019
Epoch 6/10
2251/2251 - 2s - loss: 0.0019
Epoch 7/10
2251/2251 - 2s - loss: 0.0019
Epoch 8/10
2251/2251 - 2s - loss: 0.0019
Epoch 9/10
2251/2251 - 2s - loss: 0.0019
Epoch 10/10
2251/2251 - 2s - loss: 0.0019
Epoch 1/10
2250/2250 - 3s - loss: 0.0024
Epoch 2/10
2250/2250 - 2s - loss: 0.0018
Epoch 3/10
2250/2250 - 2s - loss: 9.7867e-04
Epoch 4/10
2250/2250 - 2s - loss: 5.5258e-04
Epoch 5/10
2250/2250 - 2s - loss: 4.2387e-04
Epoch 6/10
2250/2250 - 2s - loss: 4.0128e-04
Epoch 7/10
2250/2250 - 2s - loss: 3.9388e-04
Epoch 8/10
2250/2250 - 2s - loss: 3.9263e-04
Epoch 9/10
2250/2250 - 2s - loss: 4.0170e-04
Epoch 10/10
2250/2250 - 2s - loss: 3.9045e-04
Epoch 1/10
2251/2251 - 3s - loss: 0.0037
Epoch 2/10
2251/2251 - 2s - loss: 0.0022
Epoch 3/10
2251/2251 - 2s - loss: 8.4107e-04
Epoch 4/10
2251/225

In [290]:
from sklearn.metrics import mean_squared_error
for i in range(6):

  trainPredict = model_list[i].predict(trainX_list[i])
  testPredict = model_list[i].predict(testX_list[i])
  #invert predictions
  trainPredict = scaler.inverse_transform(trainPredict)
  
  trainY = scaler.inverse_transform([trainY_list[i]])
  testPredict = scaler.inverse_transform(testPredict)

  testY = scaler.inverse_transform([testY_list[i]])
  #print(test)
  # calculate root mean squared error
  trainScore = (mean_squared_error(trainY[0], trainPredict[:,0]))
  print(place_name[i])
  print('Train Score: %.2f RMSE' % (trainScore))
  testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
  print('Test Score: %.2f RMSE' % (testScore))

 Tokyo
Train Score: 0.07 RMSE
Test Score: 0.12 RMSE
 Naha
Train Score: 0.01 RMSE
Test Score: 0.05 RMSE
 Sendai
Train Score: 0.02 RMSE
Test Score: 0.03 RMSE
 Osaka
Train Score: 0.02 RMSE
Test Score: 0.10 RMSE
 Nigata
Train Score: 0.04 RMSE
Test Score: 0.13 RMSE
 Fukuoka
Train Score: 0.07 RMSE
Test Score: 0.21 RMSE
